<a href="https://colab.research.google.com/github/avner8943/bootcamp/blob/master/decision_tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np

def split(data, feature, split_val):

  right = data[data[feature] >= split_val]

  left = data[data[feature] < split_val]

  return left, right


def calc_gini(left, right, classes):

  tot_counts = float(len(left) + len(right))

  gini = 0.0

  left_0 = np.count_nonzero(left[:,-1] == classes[0])
  left_1 = left.shape[0] - left_0

  right_0 = np.count_nonzero(right[:,-1] == classes[0])
  right_1 = right.shape[0] - right_0

  gini = (1 - left_0**2+left_1**2)*(left.shape[0]/tot_counts) + (1 - right_0**2+right_1**2)*(right.shape[0]/tot_counts)

  return gini



def get_split(data):

  classes = np.unique(data[:,-1])

  best_idx, best_val, best_gini_score, best_groups = 1e3,1e3,1e3,0

  for index in range(len(dataset[0])-1):
    for row in dataset:
      groups = test_split(index, row[index], dataset)
      gini = gini_index(groups, class_values)
      print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
      if gini < b_score:
        b_index, b_value, b_score, b_groups = index, row[index], gini, groups
	return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
dataset = [[2.771244718,1.784783929,0],
	[1.728571309,1.169761413,0],
	[3.678319846,2.81281357,0],
	[3.961043357,2.61995032,0],
	[2.999208922,2.209014212,0],
	[7.497545867,3.162953546,1],
	[9.00220326,3.339047188,1],
	[7.444542326,0.476683375,1],
	[10.12493903,3.234550982,1],
	[6.642287351,3.319983761,1]]
split = get_split(dataset)
print('Split: [X%d < %.3f]' % ((split['index']+1), split['value']))



In [7]:
# Split a dataset based on an attribute and an attribute value
def test_split(index, value, dataset):
	left, right = list(), list()
	for row in dataset:
		if row[index] < value:
			left.append(row)
		else:
			right.append(row)
	return left, right

dataset = [[2.771244718,1.784783929,0],
	        [1.728571309,1.169761413,0],
	        [3.678319846,2.81281357,0],
          [3.961043357,2.61995032,0],
          [2.999208922,2.209014212,0],
          [7.497545867,3.162953546,1],
          [9.00220326,3.339047188,1],
          [7.444542326,0.476683375,1],
          [10.12493903,3.234550982,1],
          [6.642287351,3.319983761,1]]
    
for index in range(len(dataset[0])-1):
  for row in dataset:
    groups = test_split(index, row[index], dataset)
    print([len(group) for group in groups])


print('a')

[1, 9]
[0, 10]
[3, 7]
[4, 6]
[2, 8]
[7, 3]
[8, 2]
[6, 4]
[9, 1]
[5, 5]
[2, 8]
[1, 9]
[5, 5]
[4, 6]
[3, 7]
[6, 4]
[9, 1]
[0, 10]
[7, 3]
[8, 2]
a


In [21]:
a = np.array([[  1, 275],
       [  1, 441],
       [  2, 494],
       [  1, 593],
       [  2, 679],
       [  2, 533],
       [  2, 686],
       [  3, 559],
       [  1, 219],
       [  3, 455],
       [  4, 605],
       [  4, 468],
       [  3, 692],
       [  4, 613]])


a = np.append(a,np.arange(14).reshape(14,-1), axis=1)

print(a)

print(np.split(a[:,1:], np.unique(a[:, 0], return_index=True)[1][1:]))


a.sort(axis=0)
print(np.split(a[:,1:], np.unique(a[:, 0], return_index=True)[1][1:]))




[[  1 275   0]
 [  1 441   1]
 [  2 494   2]
 [  1 593   3]
 [  2 679   4]
 [  2 533   5]
 [  2 686   6]
 [  3 559   7]
 [  1 219   8]
 [  3 455   9]
 [  4 605  10]
 [  4 468  11]
 [  3 692  12]
 [  4 613  13]]
[array([[275,   0],
       [441,   1]]), array([[494,   2],
       [593,   3],
       [679,   4],
       [533,   5],
       [686,   6]]), array([[559,   7],
       [219,   8],
       [455,   9]]), array([[605,  10],
       [468,  11],
       [692,  12],
       [613,  13]])]
[array([[219,   0],
       [275,   1],
       [441,   2],
       [455,   3]]), array([[468,   4],
       [494,   5],
       [533,   6],
       [559,   7]]), array([[593,   8],
       [605,   9],
       [613,  10]]), array([[679,  11],
       [686,  12],
       [692,  13]])]
